# Projet - Message cachés
Le but de ce projet est de transmettre un message chiffré et caché dans une image toute banale.

## Partie B : Stéganographie
Nous allons faire apparaître de l'espace mémoire dans une image sans que personne ne s'en rend compte !

### Quatrième étape : 
**Bits de poids fort, bits de poids faible**

Petit rappel sur le codage binaire.<br />
Un nombre entier entre 0 et 255 peut-être codé à l’aide d’un octet sous la forme d’une suite de 0 et/ou de 1. Chacun de ses 0 ou 1 correspond à une puissance de 2. Soit on la prend (1) soit on la laisse (0).
Par exemple avec le nombre 204 :<br />

|Les puissances| $2^7$| $2^6$| $2^5$ | $2^4$ | $2^3$ | $2^2$ | $2^1$ | $2^0$|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|
|Les bits|1|1|0|0|1|1|0|0|

$1\times 2^7 + 1\times 2^6 + 1\times 2^3 + 1\times 2^2 = 128 + 64 + 8 + 4 = 204$ <br />

`1100 1100` vaut donc 204. <br />
Plus un bit est à gauche, plus il correspond à un grand nombre (128, 64, ...), on les appelle les bits de **poids fort**. <br />
Plus un bit est à droite, plus il correspond à un petit nombre (4, 2, 1, ...), on les appelle les bits de **poids faible**. <br />
Si lors d’un codage ou d’une transmission, on fait une erreur sur un bit de poids faible, cette erreur ne sera pas trop importante. <br />
Ainsi, si j’écris `1100 1000` (un bit erroné), mon résultat est de 200. Ce qui est assez proche de 204.<br />
Par contre, si j’écris `1000 1100` (un autre bit erroné), mon résultat est de 140 qui est très éloigné de 204.<br />

Le principe de la stéganographie numérique est de dire que les bits de poids faible sont sans importance. On peut
les enlever. En effet, si je décide de supprimer les 3 bits de poids faible correspondants aux puissances $2^2$ , $2^1$ et $2^0$ je vais commettre au maximum une erreur de $2^2 + 2^1 + 2^0 = 7$. <br />

J’obtiens alors 3 bits libres lesquels je vais pouvoir dissimuler mon message !<br /><br /><br />

**Un masque**

Nous avons déjà vu dans les opérateurs binaires, si, si ! Rappelez-vous ! <br />
Ce sont les portes `AND`, `OR`, `XOR`, `NOT`, ... <br />
En utilisant une de ses portes nous allons pouvoir créer ce qu'on appelle un masque, qui nous permettra d'effacer les bits de poids faibles sans trop d'efforts. <br />
Voici quelques tables de vérité pour mémoire :

|AND|0|1|
|:--|:--|:--|
|0|0|0|
|1|0|1|

|OR|0|1|
|:--|:--|:--|
|0|0|1|
|1|1|1|

|XOR|0|1|
|:--|:--|:--|
|0|0|1|
|1|1|0|

|NOT|0|1|
|:--|:--|:--|
||1|0|

***Exercice 1 :***
Quel est le résultat de `12 AND 7` ? <br />
"*Facile*", il suffit de de convertir les nombres en binaires et d'effectuer bit par bit l'opération `AND`. <br />
Autrement dit : <br />

||12| vaut| 1100|
|-|-|-|-|
||7 |vaut |0111|
|AND|||0100|

Donc `12 AND 7` vaut `4` !

On peut le vérifier avec un petit code Python puisque le `AND` binaire s'écrit `&`.

In [ ]:
12 & 7

***Exercice 2 :*** 
Si je veux effectuer un masque sur les 3 premiers bits de poids faibles sur le nombre 12, quel opérateur dois-je utiliser ? <br />
"*Facile à nouveau*", c'est `12 AND NOT 7`.... :-( <br />
Oui, bon, j'avoue ce n'était pas si évident. Voyons plutôt : <br />

||12| vaut| 1100|
|-|-|-|-|
|| NOT 7 |vaut |1000|
|AND|||1000|

J'ai donc bien réussi à faire "disparaître" les 3 premiers bits de poids faibles de mon nombre 12.

Vérifions avec un code Python (en binaire le `NOT` s'écrit `~`).

In [ ]:
12 & ~ 7

***Exercice 3:*** Écrire une fonction `espaces(nb,bt)` qui prend en arguments deux entiers `nb` et `bt`.<br />
* `nb` est un entier entre 0 et 255.
* `bt` est entier entre 0 et 8.
La fonction transforme le nombre `nb` en remplaçant ses `bt` bits de poids faible par des 0.

In [ ]:
def espaces(nb, bt):
    '''
    Entrées :
    nb est un nombre entier entre 0 et 255
    bt est un nombre entier entre 0 et 8
    Sortie :
    un nb entier
    Rôle :
    on supprime les bt bits de poids faible à l’aide d’un masque
    '''
    space = 0
    for i in range(bt):
        space = space + 2**i
    return nb & ~ space

In [ ]:
# on teste la fonction
espaces(12,3) # doit renvoyer 8

In [ ]:
# on teste à nouveau la fonction
espaces(204,6) # doit renvoyer 192

<br /><br /><br /><br /><br /><br /><br /><br />

**Des images**
Nous allons maintenant travailler sur des images

Pour commencer, nous devons télécharger un module spécifique, le module `Pillow`

In [ ]:
import sys
!{sys.executable} -m pip install pillow

Nous pouvons alors importer le module dans notre notebook et travailler avec son sous module `Image`.

In [ ]:
from PIL import Image

Nous avons maintenant accès à toute une série de méthode qui nous permettent de travailler sur des images. <br />
Voici quelques exemples :

In [ ]:
# pour charger une image
img = Image.open('couleurs.bmp')
# pour afficher l'image dans le notebook
display(img)
# pour afficher l'image dans un IDE (comme Pyzo)
# img.show()

# pour récupérer les dimensions de l'image
largeur, hauteur = img.size
print('Nombre de pixels en largeur :', largeur)
print('Nombre de pixels en hauteur :', hauteur)

# pour récupérer les composantes Rouge, Vert et Bleu d'un pixel (là ,j'ai pris le pixel de coordonnées (50, 150))
R, V, B = img.getpixel((50, 150))
print('La composante de rouge est du pixel (50,150):\t', R)
print('La composante de vert est du pixel (50,150):\t', V)
print('La composante de bleu est du pixel (50,150):\t', B)

# pour modifier la composante Rouge, Vert et Bleu d'un pixel
# img.putpixel((x, y),(R, V, B))

# pour modifier tous les pixels d'une image
# par exemple ici pour simuler la vision d'un daltonien
for x in range(largeur):
    for y in range(hauteur):
        # on prend la couleur
        R, V, B = img.getpixel((x, y))
        moyenneRV = (R+V)//2
        # on modifie la conleur
        img.putpixel((x, y),(moyenneRV, moyenneRV, B))

# pour sauvegarder l'image modifiée
img.save("nouvelle_image.jpg")
# et afficher le nouveau résultat
display(img)

# on ferme le fichier proprement
img.close()

***Exercice 4:*** Ecrire une fonction `image_faible(nom,bt,verbose)` qui prend en argument `nom` le nom de l'image complet (avec son extension) et `bt` un entier entre 0 et 8 qui correspond au nombre de bits de poids faibles que nous allons annuler sur chaque composante de rouge, vert et bleu et enfin `verbose`, un booléen, qui indique si onsouhaite avoir ou non des commentaires sur le travail effectué par nore fonction.

In [ ]:
def image_faible(nom,bt, verbose=False):
    '''
    Entrées :
    nom est de type string, c'est le nom de l'image avec son extension
    bt est un nombre entier entre 0 et 8
    verbose est un booléen qui par défaut faux False. S'il est vrai, la fonction commentera ses résultats
    Sortie :
    aucune
    Rôle : 
    on supprime les bt bits de poids faible de chaque composante rouge, vert, bleu de chaque pixel
    '''
    photo = Image.open(nom)
    largeur , hauteur = photo.size
    for x in range(largeur):
        for y in range(hauteur):
            rouge , vert , bleu = photo.getpixel((x,y))
            # on supprime les bits de poids faible
            rouge = espaces(rouge,bt) 
            vert = espaces(vert,bt)
            bleu = espaces(bleu,bt)
            # on réinjecte la nouvelle composante
            photo.putpixel((x,y),(rouge,vert,bleu))
    # un peu de commentaire
    if verbose == True :
        print('On a annulé',bt,'bits de poids faible')
    # on sauvegarde le résultat
    photo.save('poids_faible_'+str(bt)+'.bmp')
    # on affiche le résultat
    # version Notebook
    display(photo)
    # version IDE (comme Pyzo)
    # photo.show() 
    # on ferme fichier
    photo.close()

Maintenant que nous avons cette fonction, nous allons pouvoir tester pour chaque valeur de `bt`l'impact sur la qualité de l'image.

In [ ]:
for bt in range(0,9):
    image_faible('couleurs.bmp',bt, True)

Au regard de ces résultats, quelle est la valeur de `bt` optimale ?

*Réponse :* La valeur optimale semble être de 3 bits. L'image n'est pas vraiment dégradée.<br/>
Nous créeons ainsi, pour chaque pixel 3 bits disponible pour le rouge, 3 pour le vert, 3 pour le bleu, soit 9 bits. <br />
Cette image a $450\times450 = 20~250$ pixels soit $20~250\times 9 =1~822~500$ bits disponibles. <br />
(soit environ 1780 ko) <br />

Comme 1 caractère est codé sur 6 bits (cf Notebook précédent), nous allons pouvoir caché un texte long de $303~750$ caractères. (sans les espaces et ponctuations car nous avions décidé de les supprimer !)

***Question subsidaire***

Peut-on faire ce travail avec n'importe quelle image ?????
<br />
<br />
<br />
<br />
<br />

### Cinquième étape : 
Nous avons récupéré plein de bits dans les bits de poids faibles de notre image. Ils sont maintenant disponibles pour y cacher notre message. <br />
Mais rappelez-vous nous avions écrit un petit programme qui permet de converit un texte (sans accent, sans ponctuation) en une suite de 0 et de 1.

In [ ]:
# le dictionnaire pour avoir une correspondance entre les caractères et les suites de 0 et de 1
code_dico = {
    'a':'000000', 
    'b':'000001',
    'c':'000010',
    'd':'000011',
    'e':'000100',
    'f':'000101',
    'g':'000110',
    'h':'000111',
    'i':'001000',
    'j':'001001',
    'k':'001010',
    'l':'001011',
    'm':'001100',
    'n':'001101',
    'o':'001110',
    'p':'001111',
    'q':'010000',
    'r':'010001',
    's':'010010',
    't':'010011',
    'u':'010100',
    'v':'010101',
    'w':'010110',
    'x':'010111',
    'y':'011000',
    'z':'011001',
    '0':'011010',
    '1':'011011',
    '2':'011100',
    '3':'011101',
    '4':'011110',
    '5':'011111',
    '6':'100000',
    '7':'100001',
    '8':'100010',
    '9':'100011',
    'fin':'100100'
}

# la fonction pour convertir en binaire
def conversion_binaire(texte, code_dico):
    message = ''
    for caractere in texte :
        message = message + code_dico[caractere]
    message = message + code_dico['fin']
    return message

# la fonction pour retrouver le message originel
def conversion_chaine(texte_binaire, code_dico):
    message =''
    paquet ='' 
    for bit in texte_binaire :
        # faire un paquet de 6
        paquet = paquet + bit
        if len(paquet) == 6:
            # si c'est un paquet de 6 retrouver la clef dans le dictionnaire code_dico
            for clef, valeur in code_dico.items():
                if valeur == paquet:
                    message = message + clef
            paquet=''
    return message

Nous aurons aussi besoin de convertir une chaîne de caractère contenant uniquement des 0 et des 1 en un nombre décimale. Par exemple `'110'` correspond au nombre binaire `110` et doit être converti en `6`.

In [ ]:
def bin_to_decimal(chaine_bin):
    taille = len(chaine_bin)
    decimale = 0
    for i in range(taille-1,-1,-1):
        # boucle POUR avec l'index i décroissant
        decimale = decimale + int(chaine_bin[i])*2**(taille-i-1)
    return decimale

In [ ]:
# pour tester
bin_to_decimal('110')

Notre travail va donc consister à prendre une image, créer des bits "vides" sur les 3 premiers bits de poids faibles du rouge, du vert et du bleu de chaque pixel puis de les remplacer par les bits de notre texte que l'on souhaite caché. <br />
Pour cela, on devra récupérer les bits du texte à cacher par paquet de 3. <br />
Ensuite pour les rajouter les bits sur les composantes de rouge, vert et bleu, un simple opérateur binaire `OR` suffira. Pour mémoire en Python le `OR` binaire s'écrit `|`.

Pour rajouter notre paquet à une des composantes, nous pouvons utiliser la fonction suivante :

In [ ]:
def rajouter_paquet(texte_bin, composante, i):
    # on doit créer un paquet de longueur 3
    paquet =''
    while i<len(texte_bin) and len(paquet)<3:
        paquet = paquet + texte_bin[i]
        i = i + 1
    # on convertit la suite de 0 et de 1 en décimale
    paquet_decimale = bin_to_decimal(paquet)
    # on rajoute ce paquet au rouge
    return composante | paquet_decimale, i

In [ ]:
def cache_message(image, texte, code_dico):

    #on convertit le texte en binaire
    texte_bin = conversion_binaire(texte, code_dico)
    #index qui indique la poistion de lecture du texte_bin
    i = 0
    
    # on crée les "trous" dans l'image pour les compléter
    photo = Image.open(image)
    largeur , hauteur = photo.size
    for x in range(largeur):
        for y in range(hauteur):
            if i<len(texte_bin) : #sinon, on a terminé, ce n'est pas la peine de continuer
                # on récupère la composante de rouge, vert et bleu de l'image
                rouge , vert , bleu = photo.getpixel((x,y))
                # on supprime les 3 bits de poids faible
                rouge = espaces(rouge,3) 
                vert = espaces(vert,3)
                bleu = espaces(bleu,3)

                # nous avons créé 3 espaces de 3 bits, on va donc créer 3 paquets pris dans le texte
                rouge, i = rajouter_paquet(texte_bin, rouge, i)
                bleu, i = rajouter_paquet(texte_bin, bleu, i)
                vert, i = rajouter_paquet(texte_bin, vert, i)       

                # on réinjecte la nouvelle composante
                photo.putpixel((x,y),(rouge,vert,bleu))
                
            elif i < len(texte_bin) + 6:
                # on a terminé, on met le code de fin
                # on récupère la composante de rouge, vert et bleu de l'image
                rouge , vert , bleu = photo.getpixel((x,y))
                # on supprime les 3 bits de poids faible
                rouge = espaces(rouge,3) 
                vert = espaces(vert,3)
                bleu = espaces(bleu,3)

                # nous avons créé 3 espaces de 3 bits, on va donc créer 3 paquets pris dans le texte
                rouge, i = rajouter_paquet('100', rouge, i)
                bleu, i = rajouter_paquet('100', bleu, i)      

                # on réinjecte la nouvelle composante
                photo.putpixel((x,y),(rouge,vert,bleu))
                # le paquet de fin est donc toujours '100100000'
            
            # pas de else, car sinon, on ne fait rien et on gagne du temps en traitement
                
    # on sauvegarde le résultat
    photo.save('message_cache_texte.bmp')
    # on affiche le résultat
    # version Notebook
    display(photo)
    # version IDE (comme Pyzo)
    # photo.show() 
    # on ferme fichier
    photo.close()

In [ ]:
# on cache notre message
cache_message('couleurs.bmp', 'ilnyenapaspourcommencermaisjepensequejepeuxentrouverunrapidement', code_dico)

### Sixième étape : 
Il ne nous reste plus qu'à retrouver un message caché dans une image. On a besoin pour chaque composante rouge, vert ou bleu, de récupérer les 3 bits de poids faibles pour les rajouter à notre texte écrit en binaire.

In [ ]:
def creer_paquet(composante):
    # conversion en binaire en ne gardant que les 3 derniers bits
    for i in range(0,5):
        if composante >= 2**(7-i):
            # on supprimer la puissance de 2 correspondante (c'est un bit de poids fort)
            composante = composante - 2**(7-i)
    paquet =''
    for i in range(5,8):
        if composante >= 2**(7-i):
            # on supprimer la puissance de 2 correspondante (c'est un des bits de poids faible)
            composante = composante - 2**(7-i)
            paquet = paquet + '1'
        else :
            paquet = paquet + '0'
    return paquet

In [ ]:
# on teste, par exemple creer_paquet(209) doit renvoyer '1'
creer_paquet(209)

Le problème reste de savoir quand on aura terminer de lire le message. On a donc créé un paquet de fin `'100100000'`.

In [ ]:
def retrouver_message(image, code_dico):
    
    message_bin = ''
    paquet =''
    
    photo = Image.open(image)
    largeur , hauteur = photo.size
    for x in range(largeur):
        for y in range(hauteur):
            if paquet != '100100000' : #sinon, on a terminé, ce n'est pas la peine de continuer
                message_bin = message_bin + paquet
                
                # on récupère la composante de rouge, vert et bleu de l'image
                rouge , vert , bleu = photo.getpixel((x,y))
                paquet = ''
                # on lit uniquement les 3 bits de poids faible de chaque composante
                paquet = paquet + creer_paquet(rouge)
                paquet = paquet + creer_paquet(vert)
                paquet = paquet + creer_paquet(bleu)
    
    # on ferme fichier image
    photo.close()
    
    # on convertit le message pour qu'il soit lisible
    message = conversion_chaine(message_bin, code_dico)
    return message

In [ ]:
retrouver_message('message_cache_texte.bmp', code_dico)